<a href="https://colab.research.google.com/github/GdGato/Fast_IA/blob/339/ACELERATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install accelerate -U

In [ ]:
from fastai.distributed import *
from fastai.vision.all import *
from fastai.vision.models.xresnet import *

from accelerate import notebook_launcher
from accelerate.utils import write_basic_config

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


False

In [ ]:
path = untar_data(URLs.PETS)

In [ ]:
files = get_image_files(path/"images")
len(files)

7390

In [ ]:
# Función para obtener la etiqueta de la imagen
def label_func(f): return f[0].isupper()

In [ ]:
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))

In [ ]:
path_modelo=('/content/drive/MyDrive/Almunos/mascotas.pth')
learn = load_learner(path_modelo)

In [ ]:
learn.dls = dls

In [ ]:
def get_y(o): return o[0].isupper()
def train(path):
    dls = ImageDataLoaders.from_name_func(
        path, get_image_files(path), valid_pct=0.2,
        label_func=get_y, item_tfms=Resize(224))

In [ ]:
def train(path):
    dls = ImageDataLoaders.from_name_func(
        path, get_image_files(path), valid_pct=0.2,
        label_func=get_y, item_tfms=Resize(224))
    learn = vision_learner(dls, resnet34, metrics=error_rate).to_fp16()
    with learn.distrib_ctx(sync_bn=False, in_notebook=True):
        learn.fine_tune(1)

In [ ]:
notebook_launcher(train, (path,), num_processes=2)

Launching training on one GPU.
Training Learner...


epoch,train_loss,valid_loss,error_rate,time
0,0.388660,0.243004,0.107578,01:26


epoch,train_loss,valid_loss,error_rate,time
0,0.241967,0.174976,0.079161,01:22


In [ ]:
 # Exportar el modelo entrenado
learn.export('/content/drive/MyDrive/Almunos/mascotas_v1.pth')